<a href="https://colab.research.google.com/github/dentadelta/123/blob/master/Parallel_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sentence1input = [1,2,3]
sentence2input = [4,5,6]
sentence1output = [7,8,9]
sentence2output = [10,11,12]
batch_input = [sentence1input, sentence2input]
batch_input = torch.tensor(batch_input)
print(batch_input)

tensor([[1, 2, 3],
        [4, 5, 6]])


In [ ]:
h = 8 # 8 parallel attention layer. -> I probaly going to use 6 layers because a mining rig consists of 6 GPU so I'm happy with running each layer in parallel
N = 6 # 6 stacks of encoder layer
dmodel = 512 #512 features
dk= int(dmodel/h) #key size, and value size
dv = int(dmodel/h)
dff = 2048 #embedding dimension
maximum_sequence_length = 3 #in real world in could be something like 128 pad pad sequence

In [ ]:
num_embedding = len(sentence1input)+len(sentence2input)+len(sentence1output)+len(sentence2output)
print('Number of vocabulary size is:', num_embedding,'\n','In real life this number is as large as the vocabulary size - say 5000 unique words') 

Number of vocabulary size is: 12 
 In real life this number is as large as the vocabulary size - say 5000 unique words


In [ ]:
# lets create an embedded layer:
embedded_layer = nn.Embedding(dff,dmodel).to('cuda:0')

In [ ]:
# lets transform the batch input
batch_input = embedded_layer(batch_input.to('cuda:0'))

In [ ]:
#lets apply positional encoding --> Borrow this function from Pytorch (because it works)
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


In [ ]:
pe = PositionalEncoding(dmodel,0.1,num_embedding).to('cuda:0')

In [ ]:
batch_input = pe(batch_input)

In [ ]:
# Lets create a self attention layer
class MultiAttentionHead(nn.Module):
  def __init__(self,dmodel,dk,dv,maximum_sequence_length):
    super(MultiAttentionHead, self).__init__()

    # This is where you split your model, knowing that everything will be converted to CPU for concat operation then put back the output to GPU:0
    # You can only use 1 GPU in google colab, thats why I only use GPU:0
    # The vanila transformer paper states that a mask layer is only an optional in an encoder layer, thats why I havent implemented

    #Layer1: GPU:0
    self.keyTransformer_1 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_1 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_1  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer2: GPU:1
    self.keyTransformer_2 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_2 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_2  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer3: GPU:2
    self.keyTransformer_3 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_3 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_3  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer4: GPU:3
    self.keyTransformer_4 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_4 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_4  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer5: GPU:4
    self.keyTransformer_5 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_5 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_5  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer6: GPU:5
    self.keyTransformer_6 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_6 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_6 = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer7: GPU:6
    self.keyTransformer_7 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_7 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_7  = nn.Linear(dmodel,dv).to('cuda:0')


    #Layer8: GPU:7
    self.keyTransformer_8 = nn.Linear(dmodel,dk).to('cuda:0')
    self.querryTransformer_8 = nn.Linear(dmodel,dk).to('cuda:0')
    self.valueTrasformer_8  = nn.Linear(dmodel,dv).to('cuda:0')



    #Weight_0 layer:
    self.W0 = nn.Linear(dmodel,dmodel).to('cuda:0')   #Size h*dv x dmodel. But since dv = dk and dk x h = dv so it's a dmodel x dmodel layer -> cuda:0


    #LayerNormalisation
    self.Add_and_Nom = nn.LayerNorm(dmodel, eps=1e-05, elementwise_affine=True).to('cuda:0')

    self.dropout = nn.Dropout(0.1).to('cuda:0')
  

  def forward(self,x):
    k1 = self.keyTransformer_1(x)
    q1 = self.querryTransformer_1(x)
    v1 = self.valueTrasformer_1(x)

    x.to('cuda:0')  #Replace cuda:0 with the correct GPU you chose for layer 2

    k2 = self.keyTransformer_2(x)
    q2 = self.querryTransformer_2(x)
    v2 = self.valueTrasformer_2(x)

    x.to('cuda:0'). #Replace cuda:0 with the correct GPU you chose for layer 3

    k3 = self.keyTransformer_3(x)
    q3 = self.querryTransformer_3(x)
    v3 = self.valueTrasformer_3(x)

    x.to('cuda:0')  #Replace cuda:0 with the correct GPU you chose for layer 4

    k4 = self.keyTransformer_4(x)
    q4 = self.querryTransformer_4(x)
    v4 = self.valueTrasformer_4(x)

    x.to('cuda:0'). #Replace cuda:0 with the correct GPU you chose for layer 5

    k5 = self.keyTransformer_5(x)
    q5 = self.querryTransformer_5(x)
    v5 = self.valueTrasformer_5(x)

    x.to('cuda:0'). #Replace cuda:0 with the correct GPU you chose for layer 6

    k6 = self.keyTransformer_6(x)
    q6 = self.querryTransformer_6(x)
    v6 = self.valueTrasformer_6(x)

    x.to('cuda:0'). #Replace cuda:0 with the correct GPU you chose for layer 7

    k7 = self.keyTransformer_7(x)
    q7 = self.querryTransformer_7(x)
    v7 = self.valueTrasformer_7(x)

    x.to('cuda:0'). #Replace cuda:0 with the correct GPU you chose for layer 8

    k8 = self.keyTransformer_8(x)
    q8 = self.querryTransformer_8(x)
    v8 = self.valueTrasformer_8(x)

    
    def calculate_head(q,k,v):
      return torch.matmul(F.softmax(torch.matmul(q,k.transpose(-2,-1))/(dk**0.5)),v)

    head_1 = calculate_head(q1,k1,v1) 
    head_1.to('cpu')                  

    head_2 = calculate_head(q2,k2,v2) 
    head_2.to('cpu')                  

    head_3 = calculate_head(q3,k3,v3) 
    head_3.to('cpu')                  

    head_4 = calculate_head(q4,k4,v4) 
    head_4.to('cpu')                  

    head_5 = calculate_head(q1,k1,v1) 
    head_5.to('cpu')                  

    head_6 = calculate_head(q2,k2,v2) 
    head_6.to('cpu')                  

    head_7 = calculate_head(q3,k3,v3) 
    head_7.to('cpu')                  

    head_8 = calculate_head(q4,k4,v4) 
    head_8.to('cpu')                   


    All_heads = torch.cat((head_1,head_2,head_3,head_4,head_5,head_6,head_7,head_8), dim=2) #in cpu

    All_heads.to('cuda:0') #Transfer back to Cuda:0 for matrix operation

    All_heads = self.W0(All_heads) # cuda:0

    x.to('cuda:0') # transfer back to cuda:0 for layer normalisation operation

    All_heads = self.Add_and_Nom(x + All_heads)  #cuda:0

    All_heads = self.dropout(All_heads) #cuda:0

    return All_heads #cuda:0

In [ ]:
class FFN(nn.Module):
  def __init__(self,dmodel,dk,dv,maximum_sequence_length,dff):
    super(FFN,self).__init__()
    # 6 stacks in total
    self.multihead_1 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_2 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_3 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_4 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_5 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)
    self.multihead_6 = MultiAttentionHead(dmodel,dk,dv,maximum_sequence_length)

    self.lin1a = nn.Linear(dmodel,dff).to('cuda:0')
    self.dropout1 = nn.Dropout(0.1).to('cuda:0')
    self.lin1b = nn.Linear(dff,dmodel).to('cuda:0')

    self.lin2a = nn.Linear(dmodel,dff).to('cuda:0')
    self.dropout2 = nn.Dropout(0.1).to('cuda:0')
    self.lin2b = nn.Linear(dff,dmodel).to('cuda:0')

    self.lin3a = nn.Linear(dmodel,dff).to('cuda:0')
    self.dropout3 = nn.Dropout(0.1).to('cuda:0')
    self.lin3b = nn.Linear(dff,dmodel).to('cuda:0')

    self.lin4a = nn.Linear(dmodel,dff).to('cuda:0')
    self.dropout4 = nn.Dropout(0.1).to('cuda:0')
    self.lin4b = nn.Linear(dff,dmodel).to('cuda:0')

    self.lin5a = nn.Linear(dmodel,dff).to('cuda:0')
    self.dropout5 = nn.Dropout(0.1).to('cuda:0')
    self.lin5b = nn.Linear(dff,dmodel).to('cuda:0')

    self.lin6a = nn.Linear(dmodel,dff).to('cuda:0')
    self.dropout6 = nn.Dropout(0.1).to('cuda:0')
    self.lin6b = nn.Linear(dff,dmodel).to('cuda:0')


  def forward(self,x):
    sublayer1 = self.multihead_1(x)
    sublayer1 = self.lin1a(sublayer1)
    sublayer1 = F.relu(sublayer1)
    sublayer1 = self.dropout1(sublayer1)
    sublayer1 = self.lin1b(sublayer1)
    x = sublayer1

    sublayer2 = self.multihead_2(x)
    sublayer2 = self.lin2a(sublayer2)
    sublayer2 = F.relu(sublayer2)
    sublayer2 = self.dropout2(sublayer2)
    sublayer2 = self.lin2b(sublayer2)
    x = sublayer2

    sublayer3 = self.multihead_3(x)
    sublayer3 = self.lin3a(sublayer3)
    sublayer3 = F.relu(sublayer3)
    sublayer3 = self.dropout3(sublayer3)
    sublayer3 = self.lin3b(sublayer3)
    x = sublayer3

    sublayer4 = self.multihead_4(x)
    sublayer4 = self.lin4a(sublayer4)
    sublayer4 = F.relu(sublayer4)
    sublayer4 = self.dropout4(sublayer4)
    sublayer4 = self.lin4b(sublayer4)
    x = sublayer4

    sublayer5 = self.multihead_5(x)
    sublayer5 = self.lin5a(sublayer5)
    sublayer5 = F.relu(sublayer5)
    sublayer5 = self.dropout5(sublayer5)
    sublayer5 = self.lin5b(sublayer5)
    x = sublayer5

    sublayer6 = self.multihead_6(x)
    sublayer6 = self.lin6a(sublayer6)
    sublayer6 = F.relu(sublayer6)
    sublayer6 = self.dropout6(sublayer6)
    sublayer6 = self.lin6b(sublayer6)

    

    return sublayer6

In [ ]:
LastEncoderLayer = FFN(dmodel,dk,dv,maximum_sequence_length,dff)

In [ ]:
LastEncoderLayer(batch_input)

tensor([[[ 0.3714,  0.5405, -0.2181,  ...,  0.4281, -0.1826, -0.0062],
         [-0.0383,  0.2671, -0.4236,  ...,  0.0989, -0.2363, -0.1850],
         [ 0.5845,  0.3287, -0.2716,  ...,  0.1759, -0.0156,  0.0555]],

        [[ 0.2852,  0.5044, -0.4051,  ...,  0.1559,  0.0020,  0.0632],
         [-0.0102,  0.1107, -0.3334,  ...,  0.0876,  0.0242,  0.1760],
         [ 0.2235,  0.1331, -0.0245,  ..., -0.0125,  0.0559, -0.1872]]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [ ]:
# Still need to make a final encoder layer to wrap everything together
# Then work on the decoder part (which is the same as an encoder with a layer mask + using key and querry from the encoder...
# Also need to add a softmax layer at the end of Decoder output to get the actual next token)